In [7]:
# Service Account Authentication Debug Notebook
# Use this to identify and solve service account issues with your admin

# Cell 1: Setup and Configuration
import sys
import os
from ldap3 import Server, Connection, ALL, NTLM, SIMPLE
from ldap3.core.exceptions import LDAPException
import traceback
import socket
from datetime import datetime

# Your AD Configuration
AD_DOMAIN = 'fsys.net'
AD_SERVER = 'FSKHDC4.fsys.net'
AD_PORT = 389
AD_USE_SSL = False
AD_BASE_DN = 'DC=fsys,DC=net'
AD_SERVICE_USER = 'svc_workforce_app'
AD_SERVICE_PASSWORD = 'fund-jRnyPVs23!'  # Replace with actual password

print("🔧 Service Account Debug Session")
print("=" * 50)
print(f"Domain: {AD_DOMAIN}")
print(f"Server: {AD_SERVER}:{AD_PORT}")
print(f"Service User: {AD_SERVICE_USER}")
print(f"Base DN: {AD_BASE_DN}")
print(f"Timestamp: {datetime.now()}")

🔧 Service Account Debug Session
Domain: fsys.net
Server: FSKHDC4.fsys.net:389
Service User: svc_workforce_app
Base DN: DC=fsys,DC=net
Timestamp: 2025-08-27 10:23:40.395933


In [8]:
# Cell 2: Test Network and Server Connection
def test_basic_connectivity():
    print("\n🔍 Step 1: Testing Basic Connectivity")
    print("-" * 40)
    
    # Test network
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)
        result = sock.connect_ex((AD_SERVER, AD_PORT))
        sock.close()
        
        if result == 0:
            print("✅ Network connectivity: SUCCESS")
        else:
            print("❌ Network connectivity: FAILED")
            return False
    except Exception as e:
        print(f"❌ Network error: {str(e)}")
        return False
    
    # Test LDAP server response
    try:
        server = Server(AD_SERVER, port=AD_PORT, use_ssl=AD_USE_SSL, get_info=ALL)
        conn = Connection(server, auto_bind=False)
        
        if conn.bind():
            print("✅ LDAP server response: SUCCESS")
            print(f"📋 Server info: {server.info.naming_contexts}")
            conn.unbind()
            return True, server
        else:
            print("❌ LDAP server response: FAILED")
            return False, None
            
    except Exception as e:
        print(f"❌ LDAP server error: {str(e)}")
        return False, None


In [3]:
server_ok, server_obj = test_basic_connectivity()


🔍 Step 1: Testing Basic Connectivity
----------------------------------------
✅ Network connectivity: SUCCESS
✅ LDAP server response: SUCCESS
📋 Server info: ['DC=fsys,DC=net', 'CN=Configuration,DC=fsys,DC=net', 'CN=Schema,CN=Configuration,DC=fsys,DC=net', 'DC=DomainDnsZones,DC=fsys,DC=net', 'DC=ForestDnsZones,DC=fsys,DC=net']


In [22]:
# Cell 3: Service Account Username Format Testing
def test_service_account_formats():
    print("\n🔍 Step 2: Testing Service Account Username Formats")
    print("-" * 50)
    
    if not server_ok:
        print("❌ Skipping - server connection failed")
        return False, None
    
    # Different username formats to test
    username_formats = [
        # Standard formats
        f"{AD_SERVICE_USER}@{AD_DOMAIN}",
        f"fsys\\{AD_SERVICE_USER}",
        f"{AD_DOMAIN}\\{AD_SERVICE_USER}",
        AD_SERVICE_USER,
        
        # Distinguished Name formats
        f"CN={AD_SERVICE_USER},CN=Users,{AD_BASE_DN}",
        f"CN={AD_SERVICE_USER},CN=Service Accounts,{AD_BASE_DN}",
        f"CN={AD_SERVICE_USER},OU=Service Accounts,{AD_BASE_DN}",
        
        # Alternative domain formats
        f"{AD_SERVICE_USER}@fsys",
        f"FSYS\\{AD_SERVICE_USER}",
    ]
    
    print(f"Testing {len(username_formats)} different username formats:")
    
    for i, username_format in enumerate(username_formats, 1):
        print(f"\n{i}. Testing: {username_format}")
        
        # Test with SIMPLE authentication (no MD4)
        try:
            conn = Connection(
                server_obj,
                user=username_format,
                password=AD_SERVICE_PASSWORD,
                authentication=SIMPLE,
                auto_bind=True
            )
            
            if conn.bind():
                print(f"   ✅ SIMPLE auth: SUCCESS")
                conn.unbind()
                return True, ('SIMPLE', username_format)
                
        except Exception as e:
            print(f"   ❌ SIMPLE auth: {str(e)}")
        
        # Test with NTLM authentication
        try:
            conn = Connection(
                server_obj,
                user=username_format,
                password=AD_SERVICE_PASSWORD,
                authentication=NTLM,
                auto_bind=True
            )
            
            if conn.bind():
                print(f"   ✅ NTLM auth: SUCCESS")
                conn.unbind()
                return True, ('NTLM', username_format)
                
        except Exception as e:
            print(f"   ❌ NTLM auth: {str(e)}")
    
    print("\n❌ ALL USERNAME FORMATS FAILED")
    return False, None

In [23]:
# Run service account format tests
service_success, service_details = test_service_account_formats()


🔍 Step 2: Testing Service Account Username Formats
--------------------------------------------------
Testing 9 different username formats:

1. Testing: svc_workforce_app@fsys.net
   ✅ SIMPLE auth: SUCCESS


In [24]:
# Cell 3: Service Account Username Format Testing
def test_service_account_formats():
    print("\n🔍 Step 2: Testing Service Account Username Formats")
    print("-" * 50)
    
    if not server_ok:
        print("❌ Skipping - server connection failed")
        return False, None
    
    # Different username formats to test
    username_formats = [
        # Standard formats
        f"{AD_SERVICE_USER}@{AD_DOMAIN}",
        f"fsys\\{AD_SERVICE_USER}",
        f"{AD_DOMAIN}\\{AD_SERVICE_USER}",
        AD_SERVICE_USER,
        
        # Distinguished Name formats
        f"CN={AD_SERVICE_USER},CN=Users,{AD_BASE_DN}",
        f"CN={AD_SERVICE_USER},CN=Service Accounts,{AD_BASE_DN}",
        f"CN={AD_SERVICE_USER},OU=Service Accounts,{AD_BASE_DN}",
        
        # Alternative domain formats
        f"{AD_SERVICE_USER}@fsys",
        f"FSYS\\{AD_SERVICE_USER}",
    ]
    
    print(f"Testing {len(username_formats)} different username formats:")
    
    for i, username_format in enumerate(username_formats, 1):
        print(f"\n{i}. Testing: {username_format}")
        
        # Test with SIMPLE authentication (no MD4)
        try:
            conn = Connection(
                server_obj,
                user=username_format,
                password=AD_SERVICE_PASSWORD,
                authentication=SIMPLE,
                auto_bind=True
            )
            
            if conn.bind():
                print(f"   ✅ SIMPLE auth: SUCCESS")
                conn.unbind()
                return True, ('SIMPLE', username_format)
                
        except Exception as e:
            print(f"   ❌ SIMPLE auth: {str(e)}")
        
        # Test with NTLM authentication
        try:
            conn = Connection(
                server_obj,
                user=username_format,
                password=AD_SERVICE_PASSWORD,
                authentication=NTLM,
                auto_bind=True
            )
            
            if conn.bind():
                print(f"   ✅ NTLM auth: SUCCESS")
                conn.unbind()
                return True, ('NTLM', username_format)
                
        except Exception as e:
            print(f"   ❌ NTLM auth: {str(e)}")
    
    print("\n❌ ALL USERNAME FORMATS FAILED")
    return False, None

In [25]:
service_success, service_details = test_service_account_formats()


🔍 Step 2: Testing Service Account Username Formats
--------------------------------------------------
Testing 9 different username formats:

1. Testing: svc_workforce_app@fsys.net
   ✅ SIMPLE auth: SUCCESS


In [28]:
# Cell 4: Service Account Permission Testing
def test_service_account_permissions():
    print("\n🔍 Step 3: Testing Service Account Permissions")
    print("-" * 50)
    
    if not service_success:
        print("❌ Skipping - service account authentication failed")
        return False
    
    auth_method, username_format = service_details
    print(f"Using working format: {username_format} ({auth_method})")
    
    try:
        # Connect with working credentials
        conn = Connection(
            server_obj,
            user=username_format,
            password=AD_SERVICE_PASSWORD,
            authentication=SIMPLE if auth_method == 'SIMPLE' else NTLM,
            auto_bind=True
        )
        
        # Test 1: Can read domain information
        print("\n📋 Test 1: Domain Information Access")
        try:
            conn.search(AD_BASE_DN, '(objectClass=domain)', attributes=['name'])
            if conn.entries:
                print("   ✅ Can read domain information")
            else:
                print("   ❌ Cannot read domain information")
        except Exception as e:
            print(f"   ❌ Domain read error: {str(e)}")
        
        # Test 2: Can search for users
        print("\n👤 Test 2: User Search Permission")
        try:
            conn.search(
                AD_BASE_DN, 
                '(objectClass=user)',
                attributes=['sAMAccountName'],
                size_limit=5
            )
            if conn.entries:
                print(f"   ✅ Found {len(conn.entries)} users")
                for entry in conn.entries[:3]:
                    print(f"      - {entry.sAMAccountName}")
            else:
                print("   ❌ No users found or no permission")
        except Exception as e:
            print(f"   ❌ User search error: {str(e)}")
        
        # Test 3: Can search for specific user (amila.g)
        print("\n🎯 Test 3: Specific User Search (amila.g)")
        try:
            conn.search(
                AD_BASE_DN,
                '(sAMAccountName=amila.g)',
                attributes=['displayName', 'mail', 'memberOf']
            )
            if conn.entries:
                entry = conn.entries[0]
                print("   ✅ Found amila.g:")
                print(f"      Display Name: {entry.displayName}")
                print(f"      Email: {entry.mail}")
                print(f"      Groups: {len(entry.memberOf.values) if entry.memberOf else 0} groups")
            else:
                print("   ❌ amila.g not found")
        except Exception as e:
            print(f"   ❌ Specific user search error: {str(e)}")
        
        # Test 4: Can search for groups
        print("\n👥 Test 4: Group Search Permission")
        try:
            conn.search(
                AD_BASE_DN,
                '(objectClass=group)',
                attributes=['name', 'sAMAccountName'],
                size_limit=5
            )
            if conn.entries:
                print(f"   ✅ Found {len(conn.entries)} groups")
                for entry in conn.entries[:3]:
                    print(f"      - {entry.sAMAccountName}")
            else:
                print("   ❌ No groups found or no permission")
        except Exception as e:
            print(f"   ❌ Group search error: {str(e)}")
        
        # Test 5: Can search for your specific groups
        print("\n🎯 Test 5: Application Groups Search")
        app_groups = ['WPA_Admins', 'WPA_Analysts', 'WPA_Users']
        
        for group_name in app_groups:
            try:
                conn.search(
                    AD_BASE_DN,
                    f'(sAMAccountName={group_name})',
                    attributes=['name', 'member']
                )
                if conn.entries:
                    entry = conn.entries[0]
                    member_count = len(entry.member.values) if entry.member else 0
                    print(f"   ✅ Found {group_name}: {member_count} members")
                else:
                    print(f"   ❌ Group {group_name} not found")
            except Exception as e:
                print(f"   ❌ Error searching {group_name}: {str(e)}")
        
        conn.unbind()
        return True
        
    except Exception as e:
        print(f"❌ Permission test error: {str(e)}")
        return False

In [29]:
# Run permission tests
permissions_ok = test_service_account_permissions()


🔍 Step 3: Testing Service Account Permissions
--------------------------------------------------
Using working format: svc_workforce_app@fsys.net (SIMPLE)

📋 Test 1: Domain Information Access
   ✅ Can read domain information

👤 Test 2: User Search Permission
   ✅ Found 5 users
      - FSKHDC4$
      - Guest
      - fsysadmin

🎯 Test 3: Specific User Search (amila.g)
   ✅ Found amila.g:
      Display Name: Amila G
      Email: amila.g@forlagssystem.se
      Groups: 5 groups

👥 Test 4: Group Search Permission
   ✅ Found 5 groups
      - Network Configuration Operators
      - Performance Monitor Users
      - Incoming Forest Trust Builders

🎯 Test 5: Application Groups Search
   ✅ Found WPA_Admins: 1 members
   ✅ Found WPA_Analysts: 1 members
   ✅ Found WPA_Users: 1 members


In [30]:
# Cell 5: Alternative Search Base Testing
def test_alternative_search_bases():
    print("\n🔍 Step 4: Testing Alternative Search Bases")
    print("-" * 50)
    
    if not service_success:
        print("❌ Skipping - service account authentication failed")
        return
    
    auth_method, username_format = service_details
    
    # Alternative search bases to test
    search_bases = [
        AD_BASE_DN,  # DC=fsys,DC=net
        f"CN=Users,{AD_BASE_DN}",  # CN=Users,DC=fsys,DC=net
        f"OU=Förlagssystem,{AD_BASE_DN}",  # From your original config
        "",  # Root DSE
        "CN=Configuration,DC=fsys,DC=net",
        "CN=Schema,CN=Configuration,DC=fsys,DC=net"
    ]
    
    try:
        conn = Connection(
            server_obj,
            user=username_format,
            password=AD_SERVICE_PASSWORD,
            authentication=SIMPLE if auth_method == 'SIMPLE' else NTLM,
            auto_bind=True
        )
        
        for base in search_bases:
            print(f"\n📁 Testing search base: '{base}'")
            
            try:
                conn.search(
                    search_base=base,
                    search_filter='(objectClass=user)',
                    attributes=['sAMAccountName'],
                    size_limit=3
                )
                
                if conn.entries:
                    print(f"   ✅ Found {len(conn.entries)} users")
                    for entry in conn.entries:
                        print(f"      - {entry.sAMAccountName}")
                else:
                    print("   ⚠️ No users found in this base")
                    
            except Exception as e:
                print(f"   ❌ Search failed: {str(e)}")
        
        conn.unbind()
        
    except Exception as e:
        print(f"❌ Alternative search base test error: {str(e)}")

In [31]:
# Run alternative search base tests
test_alternative_search_bases()


🔍 Step 4: Testing Alternative Search Bases
--------------------------------------------------

📁 Testing search base: 'DC=fsys,DC=net'
   ✅ Found 3 users
      - FSKHDC4$
      - Guest
      - fsysadmin

📁 Testing search base: 'CN=Users,DC=fsys,DC=net'
   ✅ Found 3 users
      - Guest
      - fsysadmin
      - krbtgt

📁 Testing search base: 'OU=Förlagssystem,DC=fsys,DC=net'
   ✅ Found 3 users
      - u1100jb
      - u1008je
      - Storesafe

📁 Testing search base: ''
   ⚠️ No users found in this base

📁 Testing search base: 'CN=Configuration,DC=fsys,DC=net'
   ⚠️ No users found in this base

📁 Testing search base: 'CN=Schema,CN=Configuration,DC=fsys,DC=net'
   ⚠️ No users found in this base
